# Perfilamiento sobre algoritmo Simplex

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [1]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3099.903
BogoMIPS:            4999.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

In [2]:
%%bash
sudo lshw -C memory

  *-memory
       description: System memory
       physical id: 0
       size: 30GiB


In [3]:
import math
import time
import SimplexC
import Simplex
import numpy as np
#from pytest import approx
from scipy.optimize import linprog

**Módulo time**

Ejemplo Maximización para evualuar tiempo de ejecución

In [4]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Solution for x vector, optimization value and status:
Simplex algorithm tomó 0.0009770393371582031 segundos


In [5]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Solution for x vector, optimization value and status:
Simplex algorithm tomó 0.0009982585906982422 segundos


Con Scipy

In [6]:
c_scipu=[-3, -5]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Scipy  tomó 0.004894733428955078 segundos


Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

**Comando Magic %time**

In [7]:
%time problema.solve()

Solution for x vector, optimization value and status:
CPU times: user 532 µs, sys: 167 µs, total: 699 µs
Wall time: 501 µs


([0, 0], 0, 0)

**Cprofile**

Para poder visualizar en que secciones de código se tarda más

In [8]:
import cProfile

In [9]:
cprof = cProfile.Profile()
cprof.enable()
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

Solution for x vector, optimization value and status:
         284 function calls (282 primitive calls) in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-9-9a4092ce7cfb>:4(<module>)
        2    0.000    0.000    0.000    0.000 iostream.py:384(write)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
    16/14    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3    0.000    0.000    0.000    0.000 socket.py:438(send)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 linalg.py:314(solve)
        2    0.000    0.000    0.000    0.000 tw

In [10]:
import pstats

cprof.dump_stats("Simplex_stats_cython")

In [11]:
p_simplex_stats = pstats.Stats("Simplex_stats_cython")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

Sat May  8 22:40:34 2021    Simplex_stats_cython

         284 function calls (282 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 78 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-9-9a4092ce7cfb>:4(<module>)
        2    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py:384(write)
        3    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py:195(schedule)
    16/14    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3    0.000    0.000    0.000    0.000 /usr/local/lib/python

In [12]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

Sat May  8 22:40:34 2021    Simplex_stats_cython

         284 function calls (282 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 78 to 6 due to restriction <'solve|module'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-9-9a4092ce7cfb>:4(<module>)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:314(solve)
        1    0.000    0.000    0.000    0.000 <ipython-input-9-9a4092ce7cfb>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-9-9a4092ce7cfb>:5(<module>)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:310(_solve_dispatcher)




In [13]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

282


In [14]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 78 to 10 due to restriction <10>

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3288(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       3    0.000    0.001  interactiveshell.py:3288(run_code)
<ipython-input-9-9a4092ce7cfb>:4(<module>)                               <-       1    0.000    0.001  {built-in method builtins.exec}
iostream.py:384(write)                                                   <-       2    0.000    0.000  <ipython-input-9-9a4092ce7cfb>:4(<module>)
iostream.py:195(schedule)                                                <-       1    0.000    0.000  iostream.py:321(_schedule_flush)
                                                                                  2    0.0

In [15]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

   Ordered by: cumulative time
   List reduced from 78 to 5 due to restriction <'print|dot|ndim|solve'>

Function                                 called...
                                             ncalls  tottime  cumtime
<__array_function__ internals>:2(solve)  ->       4    0.000    0.000  linalg.py:310(_solve_dispatcher)
                                                  4    0.000    0.000  {built-in method numpy.core._multiarray_umath.implement_array_function}
linalg.py:314(solve)                     ->       4    0.000    0.000  linalg.py:102(get_linalg_error_extobj)
                                                  8    0.000    0.000  linalg.py:107(_makearray)
                                                  4    0.000    0.000  linalg.py:112(isComplexType)
                                                  4    0.000    0.000  linalg.py:135(_commonType)
                                                  4    0.000    0.000  linalg.py:194(_assert_stacked_2d)
                 